In [1]:
%matplotlib inline
import sys, os
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['THEANO_FLAGS'] = 'device=gpu'
sys.path.append(os.getcwd()+"/..")

from tasks import check as load_filename
from scripts.imgtovideo import imgs_to_video
from data import load_data


import matplotlib.pyplot as plt
import numpy as np

import theano
import theano.tensor as T

from lasagne import layers as L

from lasagnekit.misc.plot_weights import dispims_color, tile_raster_images

from IPython.display import HTML

import pandas as pd

from tqdm import tqdm

import base64
import json

from skimage.io import imread
from skimage.transform import resize

def load_model(filename, **kw):

    model = load_filename(
        what="notebook", 
        filename=filename, 
        dataset='digits',
        **kw
    )
    return model

def build_brush_func(layers):
    
    if 'biased_out' in layers:
        bias = layers['biased_out'].b.get_value()
    elif 'bias' in layers:
        bias = layers['bias'].b.get_value()
    else:
        bias = 0
    if 'scaled_out' in layers:
        scale = layers['scaled_out'].scales.get_value()
    else:
        scale = 1
    
    X = T.tensor4()
    fn = theano.function(
        [X], 
        T.nnet.sigmoid(L.get_output(layers['brush'], X) * scale + bias)
    )
    return fn

def build_encode_func(layers):
    w = layers['output'].output_shape[2]
    X = T.tensor4()
    fn = theano.function(
        [X], 
        T.nnet.sigmoid(L.get_output(layers['coord'], X)[:, :, 0:2]) * w
    )
    return fn

def to_grid_of_images(seq_imgs):
    y = seq_imgs
    imgs = []
    for t in range(y.shape[1]):
        yy = y[:, t]
        yy = yy[:, :, :, None] * np.ones((1, 1, 1, 3))
        img = dispims_color(yy, border=0)
        imgs.append(img)
    return imgs

def seq_to_video(seq, filename='out.mp4', verbose=1, framerate=8, rate=8):
    # shape of seq should be : (examples, time, w, h) or (examples, time, 1, w, h)
    if len(seq.shape) == 5:
        seq = seq[:, :, 0, :, :]
    seq = to_grid_of_images(seq)
    seq = [np.zeros_like(seq[0])] + seq
    if os.path.exists(filename):
        os.remove(filename)
    imgs_to_video(seq, out=filename, verbose=verbose, framerate=framerate, rate=rate)

def embed_video(filename):
    video = open(filename, 'r+b').read()
    encoded = base64.b64encode(video)
    return HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                </video>'''.format(encoded.decode('ascii')))
def disp_grid(imgs, **kw):
    # shape of imgs should be : (examples, color, w, h)
    out = dispims_color(imgs.transpose((0, 2, 3, 1)) * np.ones((1, 1, 1, 3)), **kw)
    return out

/home/mcherti/miniconda/envs/databoard-env/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
Using gpu device 0: Tesla K20Xm (CNMeM is disabled, cuDNN Version is too old. Update to v5, was 3007.)
/home/mcherti/work/code/external/scikit-learn/sklearn/cross_validation.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Couldn't import dot_parser, loading of dot files will not be possible.


## Load model and build functions

In [2]:
# some selected models :
# 1) ../training/brush7/model.pkl
# 2) ../jobs/results/df2631577eaf38b002c6c0ae6f1937e1/model.pkl 
# 3) ../jobs/results/68fa4141b9f970f6bde95da53b8e54fe/model.pkl 
# 4) ../jobs/results/88b68a0c06bfb0a837cf7c0b77fe7eb9/model.pkl 
# 5) ../jobs/results/9c70a6ee9340c85b398a31f7d16be962/model.pkl
# 6) ../jobs/results/bc8b5c4156561f4f09685874d52ba20e/model.pkl
# 7) ../jobs/results/a50c86acf5a2875dda6e1ac5b8d462c1/model.pkl
# 8) ../training/brush9/model.pkl
model, data, layers, w, h, c = load_model('../training/brush11/model.pkl',
                                          #force_model_params={'w_out': 112, 'h_out': 112},
                                          kw_load_data={'include_test': True})

Loading data...


INFO:tasks:Loading data...


EOFError: 

In [ ]:
print(json.dumps(model.hypers['model_params'], indent=4))

In [ ]:
encode = build_encode_func(layers) # transforms image to sequence of coordinates

In [ ]:
brush = build_brush_func(layers) # transforms an image to sequence of images

In [ ]:
reconstruct = model.reconstruct # reconstructs an image

## Get coords from images

In [ ]:
encode(model.preprocess(data.X[0:10])).shape # (examples, time, 2)

## Create a video

In [ ]:
imgs = brush(model.preprocess(data.test.X[0:11*11])) # (examples, time, w, h)
print(imgs.shape)
seq_to_video(imgs, 'seq.mp4')
embed_video('seq.mp4')

## Reconstruct

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.imshow(disp_grid(reconstruct(model.preprocess(data.test.X[0:100]))), cmap='gray')

## iterative refinement

In [ ]:
nb_iter = 500
nb_examples = 100

np.random.seed(2)

imgs = np.empty((nb_examples, nb_iter + 1, 1, w, h)) # 1 = color channel
imgs = imgs.astype(np.float32)

imgs[:, 0] = np.random.uniform(size=(nb_examples, 1, w, h))
for i in tqdm(range(1, nb_iter + 1)):
    imgs[:, i] = reconstruct(imgs[:, i - 1])
    imgs[:, i] = imgs[:, i] > 0.5 # binarize

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.imshow(disp_grid(imgs[:, -1])) # display last time step

In [ ]:
seq_to_video(imgs, 'ir.mp4')
embed_video('ir.mp4')

## Feeding a new image

In [ ]:
strokes = load_data('strokes')

In [ ]:

# load from file

"""
img = imread('4Tbx4rALc.jpeg')
img = img[:, :, 0]
img = resize(img, (w, h), preserve_range=True)
img = img.astype(np.float32)
img /= 255.
img = 1 - img
"""

# dataset of strokes

nb = 100
dt = strokes.X[0:nb].reshape((nb, 1, w, h))

rec = dt.copy()
rec = reconstruct(rec)
#rec = rec > 0.3

fig = plt.figure(figsize=(10, 10))
plt.imshow(disp_grid(dt, bordercolor=(200, 200, 200), border=1), cmap='gray')
plt.title('original')
plt.show()

fig = plt.figure(figsize=(10, 10))
plt.imshow(disp_grid(rec,  bordercolor=(200, 200, 200), border=1), cmap='gray')
plt.title('reconstruction')
plt.show()